### Solutions

#### Question 1

Use the `pytz` and `dateutil` libraries to convert this string into a UTC naive `datetime` object.

In [1]:
t = "Feb 8, 2021 5:30pm (Denver Time)"

##### Solution

In [2]:
from dateutil import parser
import pytz

The `parser` function will not work properly as is:

In [3]:
try:
    parser.parse(t)
except parser.ParserError as ex:
    print(ex)

Unknown string format: Feb 8, 2021 5:30pm (Denver Time)


We can try a fuzzy parse:

In [4]:
parser.parse(t, fuzzy_with_tokens=True)

(datetime.datetime(2021, 2, 8, 17, 30), (' ', ' ', ' (Denver Time)'))

So now we have the naive datetime, but we'll need to localize it to Denver time:

In [5]:
dt_naive, _ = parser.parse(t, fuzzy_with_tokens=True)
dt_naive

datetime.datetime(2021, 2, 8, 17, 30)

We'll need the timezone from pytz that matches Denver. I don't know the exact timezone name, so I'll search for it in `pytz.all_timezones`:

In [6]:
[tz for tz in pytz.all_timezones if 'Denver' in tz]

['America/Denver']

So, it's `America/Denver` - let's get a pytz timezone object for it:

In [7]:
tz_denver = pytz.timezone('America/Denver')
tz_denver

<DstTzInfo 'America/Denver' LMT-1 day, 17:00:00 STD>

Next, we need to localize our datetime object with this timezone:

In [8]:
dt_aware = tz_denver.localize(dt_naive)
dt_naive, dt_aware

(datetime.datetime(2021, 2, 8, 17, 30),
 datetime.datetime(2021, 2, 8, 17, 30, tzinfo=<DstTzInfo 'America/Denver' MST-1 day, 17:00:00 STD>))

As you can see the date and time has not been changed, but we have not localized that original datetime to the Denver timezone.

Next, we need to convert the aware datetime into UTC:

In [9]:
dt_utc = dt_aware.astimezone(pytz.UTC)
dt_utc

datetime.datetime(2021, 2, 9, 0, 30, tzinfo=<UTC>)

And finally, we need to remove the timezone from `dt_utc` to make it naive:

In [10]:
dt = dt_utc.replace(tzinfo=None)
dt

datetime.datetime(2021, 2, 9, 0, 30)

Recapping all these steps:

In [11]:
t

'Feb 8, 2021 5:30pm (Denver Time)'

In [12]:
dt_naive, _ = parser.parse(t, fuzzy_with_tokens=True)
tz_denver = pytz.timezone('America/Denver')
dt_aware = tz_denver.localize(dt_naive)
dt_utc = dt_aware.astimezone(pytz.UTC)
dt = dt_utc.replace(tzinfo=None)
dt

datetime.datetime(2021, 2, 9, 0, 30)

#### Question 2

Use the `requests` library to load the following html page:

In [13]:
url = 'https://en.wikipedia.org/wiki/John_von_Neumann'

Once you have loaded that page, extract the title of that page, which is the text located between the `<title>` and `</title>` tags (often referred to as opening and closing tags, or start and end tags, respectively).

Hint: You'll want to read the Python docs for the `find` method available for strings:

https://docs.python.org/3/library/stdtypes.html?highlight=string#str.find

##### Solution

In [14]:
import requests

Let's load the page using a `GET` request:

In [15]:
page = requests.get(url)

And the page text is the following:

In [16]:
page.text

'Please set a user-agent and respect our robot policy https://w.wiki/4wJS. See also https://phabricator.wikimedia.org/T400119.\n'

We need to identify the start position of the tags `<title>` and `</title>` - then we can slice the characters between the end of the `<title>` tag and the beginning of the `</title>` tag.

In [17]:
start_open_tag = page.text.find('<title>')
start_close_tag = page.text.find('</title>')

start_open_tag, start_close_tag

(-1, -1)

Now, `start_open_tag` is the starting index of where `<title>` was found:

In [18]:
page.text[start_open_tag:start_open_tag + 50]

''

So, we actually want to start slicing right after that last `>` in the opening tag, i.e. at the start index plus the length of the tag itself:

In [19]:
page.text[start_open_tag + len('<title>'): start_open_tag + len('<title>') + 50]

' set a user-agent and respect our robot policy htt'

Of course, we have to stop the slice at the first character of the closing tag `</title>`, which is given by `start_close_tag`:

In [20]:
page.text[start_open_tag + len('<title>'): start_close_tag]

' set a user-agent and respect our robot policy https://w.wiki/4wJS. See also https://phabricator.wikimedia.org/T400119.'

#### Question 3

Use a `GET` request to this URL:

In [21]:
url = 'https://httpbin.org/json'

Use the response from that request to:
- determine the response format
- extract the response into a Python object

##### Solution

Let's issue the get request first:

In [22]:
response = requests.get('https://httpbin.org/json')

To see what the response format was we can look at the response headers:

In [23]:
response.headers

{'Server': 'awselb/2.0', 'Date': 'Thu, 11 Dec 2025 20:05:19 GMT', 'Content-Type': 'text/html', 'Content-Length': '162', 'Connection': 'keep-alive'}

In particular we are looking for the `Content-Type` header, which is `application/json`.

So we are dealing with a json response, which means we can obtain the response as a Python dictionary using the `json()` method:

In [24]:
data = response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [30]:
data

{'x': 100, 'y': 200, 'z': ['a', 'b', 'c']}

#### Question 4

Use a `POST` request to call this url:

In [31]:
url = 'https://httpbin.org/anything'

Make this call passing the following query parameters: `a=1` and `b=2`

Also, pass this dictionary as the body of the post request:

In [32]:
data = {
    'x': 100,
    'y': 200,
    'z': ['a', 'b', 'c']
}

Load the returned JSON into a Python object and print it out.

##### Solution

Let's make the request passing the `params` and `json` data:

In [33]:
response = requests.post(
    url = url,
    params={'a': 1, 'b': 2},
    json=data
)

Let's make sure the request was succesful.

In [34]:
response.status_code

503

Or, we could use the `raise_for_status()` method to raise an exception if there was an error:

In [35]:
response.raise_for_status()

HTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/anything?a=1&b=2

No exception was raised, so request was successful.

Let's see what the response content type was:

In [36]:
response.headers

{'Server': 'awselb/2.0', 'Date': 'Thu, 11 Dec 2025 20:06:07 GMT', 'Content-Type': 'text/html', 'Content-Length': '162', 'Connection': 'keep-alive'}

As we can see, the response content is json, so we can load the result into a Python dictionary:

In [37]:
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)